# Introduction

We generated some transfection results which are currently in a hand written excel file (uploaded to gdocs).
I need to reformat it to match the schema agreed upon by PSU and the DCC.

The experimental results are some bigBed files located in https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/

These bigBed files are generated from beds in the same directory.

The original versions had simple colors, but more recently we wanted to customize them so I had to override the colors that were specified in the bed files.

The root of the trackhub directory is a .git repository containing the earlier versions of the files as well.

In [1]:
import sys
sys.version
sys.path

['',
 '/home/diane/proj/encode3-curation',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/home/diane/.local/lib/python3.7/site-packages',
 '/home/diane/.local/lib/python3.7/site-packages/gcat-0.1.0-py3.7.egg',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages/IPython/extensions',
 '/home/diane/.ipython']

In [2]:
import pandas
import numpy
import itertools
import sys
import os
import collections
import pyBigWig

In [3]:
GCAT_DIR = os.path.expanduser('~diane/src/gcat')
if GCAT_DIR not in sys.path:
    sys.path.append(GCAT_DIR)
import gcat

In [4]:
autosql = '''table transfection
"class for transfection data"
(
string chrom; "GRCm38/mm10”
uint chromStart; "Start position in chromosome"
uint chromEnd; "End position in chromosome"
string name; "Name given to a region"
uint score; "Rounded average of determinations"
char[1] strand; ". for unknown"
uint thickStart; "Start of where display should be thick"
uint thickEnd; "End of where display should be thick"
uint reserved; "Used as itemRgb; color by activityBin"
string predictionCat; "prediction category"
uint count; "count of determinations"
string determ; "comma separated list of determinations as fold change relative to activity from parental vector"
float aveDeterm; "average of determinations"
float stdDeterm; "standard deviation of determinations"
string activityBin; "classification of activity; Enhancer, NotEnhancer, Threshold"
)
'''
open('/tmp/transfection.as', 'wt').write(autosql)

837

In [5]:
google_sheet = gcat.get_file('Wold_transfection_data01152017', fmt='pandas')

In [6]:
google_sheet

,#ID,Type,CHR(mm10),Start(mm10),Stop(mm10),STDEV,C2C12 Myoblast Bio Rep1,STDEV.1,C2C12 Myoblast Bio Rep2,STDEV.2,...,STDEV.3,C2C12 Myocyte Bio Rep2,STDEV.4,10T1/2 Bio Rep1,STDEV.5,10T1/2 Bio Rep2,STDEV.6,10T1/2 Mock Differentiated Bio Rep1,STDEV.7,10T1/2 Mock Differentiated Bio Rep2
0,BJW1000A,TEST ELEMENT*NESTED,chr1,134244219,134245778,0.359053,2.497751,NaN,NaN,2.132855,...,NaN,NaN,0.872601,1.755624,NaN,NaN,0.285142,1.258984,NaN,NaN
1,BJW1000B,TEST ELEMENT,chr1,134244219,134245778,0.476389,1.653045,0.198390,2.108818,1.773110,...,3.993534,11.324074,0.711102,2.082971,0.679628,2.688467,4.512507,8.622666,0.526813,1.275765
2,BJW1001,TEST ELEMENT*NESTED,chr1,134244847,134245778,0.045775,0.325529,0.058186,0.211114,4.876986,...,2.544488,14.994512,0.067867,0.265127,0.0605345,0.397979,0.084999,0.595979,0.091783,0.666264
3,BJW1002,TEST ELEMENT*TSS,chr1,134192922,134193474,0.097593,0.795625,0.074187,0.946431,0.501093,...,0.283474,1.075926,0.029145,0.603520,0.185391,1.031936,0.167062,1.264036,0.108998,1.449387
4,BJW1003,TEST ELEMENT*NESTED,chr1,134195091,134196873,0.222086,2.738967,NaN,NaN,3.394812,...,NaN,NaN,0.791119,2.942621,NaN,NaN,0.437904,2.595539,NaN,NaN
5,BJW1004,TEST ELEMENT,chr1,134195091,134196873,0.175631,0.894279,0.103021,0.490867,3.631281,...,1.574816,21.121844,0.227619,0.538779,0.0252848,0.842880,0.116144,0.798558,0.127061,1.128835
6,BJW1005,TEST ELEMENT*NESTED,chr1,134195091,134196479,0.117763,0.403462,0.037817,0.412533,2.220133,...,2.090177,14.550494,0.115628,0.333443,0.0743763,0.956223,0.142184,0.661988,0.276872,1.170382
7,BJW1006,TEST ELEMENT*NESTED,chr1,134195091,134195506,0.105161,1.710481,0.094450,0.630778,0.840585,...,0.553585,2.617453,0.286922,1.578218,0.616976,3.236429,0.099618,1.606222,0.186260,1.825527
8,BJW1007,TEST ELEMENT*NESTED,chr1,134195091,134195589,0.377482,1.691410,NaN,NaN,0.481375,...,NaN,NaN,0.389050,2.354799,NaN,NaN,0.615788,1.914250,NaN,NaN
9,BJW1008,TEST ELEMENT*NESTED,chr1,134195643,134196479,0.256856,1.114423,0.307082,1.681287,2.291935,...,0.847015,7.754665,0.142365,1.094444,0.850404,2.566563,0.081946,1.488998,0.363837,1.575713


In [7]:
transfection = pandas.read_csv('transfection-merged.csv', header=0)

In [8]:
transfection.shape

(368, 49)

In [9]:
set(google_sheet['#ID']).difference(transfection['#ID'])

{'WOLD_LR_104'}

In [10]:
set(transfection['#ID']).difference(google_sheet['#ID'])

{'WOLD2014JAN_070'}

In [11]:
annotated_transfection = google_sheet[['#ID', 'Type', 'CHR(mm10)', 'Start(mm10)', 'Stop(mm10)']].merge(transfection, left_on='#ID', right_on="#ID", how="inner", )

* Make sure ID is actually unique

In [14]:
[ (k, v) for k, v in collections.Counter(google_sheet['#ID']).items() if v > 1 ]

[]

* Add a strand column

In [15]:
def list_determs(row):
    return [str(x) for x in row if not pandas.isnull(x)]

def format_determ(row):
    return ','.join(list_determs(row))

def nan_round_to_int(x):
    if pandas.isnull(x):
        return x
    else:
        return numpy.round(x, 0)

def makebed(sheet, rep_prefix, threshold):
    activity_label = {
        True: 'Enhancer',
        False: 'NotEnhancer'
    }
    activity_color = {
        True: '65,171,93',
        False: '161,217,155'
    }
    bed = pandas.DataFrame(columns = ['chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 
                   'thickStart', 'thickEnd', 'reserved', 'predictionCat', 'count', 
                   'determ', 'aveDeterm', 'stdDeterm', 'activityBin'])

    determ_cols = [rep_prefix + '-Tech_Rep_'+x for x in ['1','2','3','4']]
    avg_col = rep_prefix + '-ASSAY'    
    bed['chrom'] = sheet['CHR(mm10)']
    bed['chromStart'] = sheet['Start(mm10)']
    bed['chromEnd'] = sheet['Stop(mm10)']
    bed['name'] = sheet['#ID']
    bed['score'] = [ nan_round_to_int(float(x * 10 )) for x in sheet[avg_col] ]
    bed['strand'] = '.'
    bed['thickStart'] = sheet['Start(mm10)']
    bed['thickEnd'] = sheet['Stop(mm10)']
    bed['reserved'] = [ activity_color[float(x) >= threshold] for x in sheet[avg_col]]
    bed['predictionCat'] = [ x.replace(' ', '_') for x in sheet['Type']]
    bed['count'] =  sheet[determ_cols].apply(lambda x: len(list_determs(x)), axis=1)
    bed['determ'] = sheet[determ_cols].apply(format_determ, axis=1)
    #bed['aveDeterm'] = sheet[avg_col]
    #bed['stdDeterm'] = sheet[rep_prefix+'-STDEV']
    bed['aveDeterm'] = sheet[determ_cols].mean(axis=1)
    bed['stdDeterm'] = sheet[determ_cols].std(axis=1)
    bed['activityBin'] = [ activity_label[float(x) >= threshold] for x in sheet[avg_col]]
    
    bed.dropna(inplace=True)
    bed['score'] = [ int(x) for x in bed['score'] ]
    return bed.sort_values(['chrom', 'chromStart'])
    

In [16]:
BED_FILE_INFO = {
    'c2c12_myoblast_biorep1.bed': ('C2C12_Myoblast_Rep1', 2.75),
    'c2c12_myoblast_biorep2.bed': ('C2C12_Myoblast_Rep2', 2.75),
    'c2c12_myocyte_biorep1.bed': ('C2C12_Myocyte_Rep1', 3.0),
    'c2c12_myocyte_biorep2.bed': ('C2C12_Myocyte_Rep2', 3.0),
    '10T_half_biorep1.bed': ('10T1/2_Rep1', 2.75),
    '10T_half_biorep2.bed': ('10T1/2_Rep2', 2.75),
    '10T_half_mock_diff_biorep1.bed': ('10T1/2_MockDiff_Rep1', 2.75),
    '10T_half_mock_diff_biorep2.bed': ('10T1/2_MockDiff_Rep2', 2.75),
}

In [17]:
def generate_bed_files(sheet, bed_file_info):
    bedframes = {}
    for filename in bed_file_info:
        rep_prefix, threshold = bed_file_info[filename]
        bed = makebed(sheet, rep_prefix, threshold)
        bedframes[filename] = bed
        bed.to_csv(os.path.join('/tmp', filename), sep='\t', header=False, index=False)
    return bedframes

Convert beds to bigBed

```for a in *.bed; do echo $a ;  /woldlab/castor/proj/programs/x86_64/bedToBigBed -as=transfection.as -type=bed9+6 $a mm10.sizes $(basename $a .bed).bigBed ; done```


import collections

scores = collections.Counter()
for filename in bedframes:
    scores.update(collections.Counter(bedframes[filename].score))
#scores

maxscore = 11

# Make wiggles

In [18]:
import pyBigWig

In [19]:
def remove_collisions(bed, verbose):
    collisions = set()
    collidedwith = set()
    last_size = None

    while last_size != len(collisions):
        last_size = len(collisions)
        filtered = bed[bed['name'].isin(collisions) == False]
        last = None    
        for i, row in filtered[['chrom', 'chromStart', 'chromEnd', 'name']].iterrows():
            if last is None or last.chrom != row.chrom:
                last = row
            else:
                if (row.chromStart >= last.chromStart) and (row.chromStart <= last.chromEnd):
                    if verbose:
                        print('|', last.chrom, '|', last.chromStart, '|', last.chromEnd, '|', 
                              last['name'], '|', row.chromStart, '|', row.chromEnd, '|', row['name'], '|', )
                    collisions.add(row['name'])
                    collidedwith.add(last['name'])
                    
                last = row
                
    return filtered, collisions, collidedwith

def fix_overlaps(bed, verbose):
    #| chr1 | 134275871 | 134276983 | UP_15 | 134276734 | 134277378 | GS_Myog_41 |
    #| chr1 | 134284098 | 134285495 | GS_Myog_39 | 134285494 | 134286451 | UP_16 |
    #| chr3 | 88138276 | 88138625 | NML_8 | 88138602 | 88139038 | NML_10 |
    #| chr6 | 108668244 | 108668981 | WOLD_LR_018 | 108668244 | 108668804 | WOLD_LR_027 |
    #| chr7 | 46313331 | 46314018 | NEG_22 | 46313389 | 46314120 | WOLD_LR_045 |
    #| chr7 | 46337340 | 46338183 | NEG_20 | 46338108 | 46338476 | GS_Myod_07 |
    #| chr8 | 123882187 | 123882932 | WOLD_LR_008 | 123882909 | 123883847 | UP_13 |
    #| chr8 | 126583914 | 126584586 | WOLD_LR_002 | 126583914 | 126584586 | WOLD_LR_002 |    
    bed = bed.copy()
    for name, new_start in [
        ('GS_Myog_41', 134276984),
        ('UP_16', 134285496),
        ('NML_10', 88138626),
    #    ('WOLD_LR_027', 108668982),
    #    ('WOLD_LR_045', 46314019),
        ('GS_Myod_07', 46338184),
    #    ('UP_13', 123882933),
    #    ('WOLD_LR_002', 126584587)
    ]:
        bed_index = bed[bed['name'] == name].index
        if len(bed_index == 1):
            i = bed_index[0]
            old_start = bed.loc[i, 'chromStart']
            assert bed.loc[i, 'name'] == name
            bed.loc[i, 'chromStart'] = new_start
            bed.loc[i, 'thickStart'] = new_start
    return bed
    


In [20]:
def read_sizes(filename):
    def alphakey(key):
        rest = key[0][3:]
        if len(rest) == 1:
            return '0' + rest
        elif not rest[1].isdigit():
            return '0' + rest
        else:
            return rest
        
    sizes = []
    for line in open(filename, 'rt'):
        records = line.strip().split()
        sizes.append((records[0], int(records[1])))
    return sorted(sizes, key=alphakey)
                     
def make_bigwig(name, bed, verbose):
    sizes = read_sizes('mm10.sizes')
    nomarked = bed[bed['predictionCat'].isin(('TEST_ELEMENT', 'CONTROL_ELEMENT'))]
    fixed = fix_overlaps(nomarked, verbose)
    filtered, collisions, collidedwith = remove_collisions(fixed, verbose)
    if verbose:
        print('filtered', len(filtered))
        print('collisions', len(collisions))
        print('colidedwith', len(collidedwith))
    with open(name + '.debug', 'wt') as outstream:
        bw = pyBigWig.open(name, 'w')
        bw.addHeader(sizes, maxZooms=10)
        #bw.addEntries(["chr1", "chr1", "chr1"], [0, 100, 125], ends=[5, 120, 126], values=[0.0, 1.0, 200.0])
        for chrom, length in sizes:
            f = filtered[filtered['chrom'].isin([chrom])]
            names = list(f['name'])
            chroms = list(f['chrom'])
            starts = list(f['chromStart'])
            ends = list(f['chromEnd'])
            values = list(f['aveDeterm'])

            if len(names) > 0:
                bw.addEntries(chroms, starts, ends=ends, values=values)
                for i in range(len(names)):
                    outstream.write('\t'.join([str(x) for x in [names[i], chroms[i], starts[i], ends[i], values[i]]]))
                    outstream.write('\n')
        bw.close()
    return filtered, collisions

def make_error_bigwig(name, bed, verbose):
    sizes = read_sizes('mm10.sizes')
    nomarked = bed[bed['predictionCat'].isin(('TEST_ELEMENT', 'CONTROL_ELEMENT'))]
    fixed = fix_overlaps(nomarked, verbose)
    filtered, collisions, collidedwith = remove_collisions(fixed, verbose)
    #bw.addEntries(["chr1", "chr1", "chr1"], [0, 100, 125], ends=[5, 120, 126], values=[0.0, 1.0, 200.0])
    order = ['chr1',  'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9',
             'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19',
             'chrX']
    bw = pyBigWig.open(name, 'w')
    bw.addHeader(sizes, maxZooms=10)
    for chrom, length in sizes:
        f = filtered[filtered['chrom'].isin([chrom])]
        names = list(f['name'])
        chroms = list(f['chrom'])
        starts = list(f['chromStart'])
        ends = list(f['chromEnd'])
        values = list((f['aveDeterm']-f['stdDeterm']).clip_lower(0))

        if len(names) > 0:
            bw.addEntries(chroms, starts, ends=ends, values=values)
    bw.close()
    return filtered, collisions

def make_bedgraph(name, bed, l=None, verbose=False):
    filtered, collisions, collidedwith = remove_collisions(
        bed[bed['predictionCat'].isin(('TEST_ELEMENT', 'CONTROL_ELEMENT'))], verbose)
    with open(name, 'wt') as outstream:
        for i, row in filtered.iterrows():
            cols = [row['chrom'], str(row['chromStart']), str(row['chromEnd']), str(row['aveDeterm'])]
            outstream.write('\t'.join(cols))
            outstream.write('\n')
            if l and l >= i:
                return
        

In [21]:
def make_bigwigs(bedframes, verbose=False):
    for filename in bedframes:
        base, ext = os.path.splitext(filename)
        bwname = os.path.join('/tmp', base + '.bw')
        if os.path.exists(bwname):
            os.unlink(bwname)
        f = make_bigwig(bwname, bedframes[filename], verbose)
        bwerror = os.path.join('/tmp', base + '-error.bw')
        f = make_error_bigwig(bwerror, bedframes[filename], verbose)


In [22]:
import sys

TH_DIR = os.path.expanduser('~/proj/trackhub')
if TH_DIR not in sys.path:
    sys.path.append(TH_DIR)

In [23]:
from trackhub import AggregateTrack, CompositeTrack, SuperTrack, Track, ViewTrack, SubGroupDefinition, default_hub

import os
from urllib.parse import urljoin

In [24]:
URLBASE='https://woldlab.caltech.edu/~diane/encode3-transfection/'
GENOME = 'mm10'

bed_attributes = {
    'c2c12_myoblast_biorep1.bed': {'cell_line': "c2", 'state': "myoblat"},
    'c2c12_myoblast_biorep2.bed': {'cell_line': "c2", 'state': "myoblat"},
    'c2c12_myocyte_biorep1.bed': {'cell_line': "c2", 'state': "myocyte"},
    'c2c12_myocyte_biorep2.bed': {'cell_line': "c2", 'state': "myocyte"},
    '10T_half_biorep1.bed': {'cell_line': "ten", 'state': "diff"},
    '10T_half_biorep2.bed': {'cell_line': "ten", 'state': 'diff'},
    '10T_half_mock_diff_biorep1.bed': {'cell_line': "ten", 'state': 'mock'},
    '10T_half_mock_diff_biorep2.bed': {'cell_line': "ten", 'state': 'mock'},
}

In [25]:
def add_psu_tracks(trackdb):
    psusuper = SuperTrack(
        name="psutrans",
        short_label="PSU trans",
        long_label="PSU Transfection tracks",
        visibility='dense',
    )

    track1 = Track(
        name="psu",
        url='https://woldlab.caltech.edu/~diane/ENCFF926ITQ.bigBed',
        tracktype='bigBed 9',
        short_label='psutrans',
        long_label='PSU Transfection',
        visibility='dense',
        itemRgb='on',
        exonNumbers="off",
        #spectrum='on',
        #scoreMin=0,
        #scoreMax=maxscore,
        )

    psusuper.add_tracks(track1)
    trackdb.add_tracks(psusuper)


In [26]:
def add_dnase_track(trackdb):
    dnase = pandas.read_excel('transfection-additional-tracks.xlsx', sheetname='DNAse')
    dnase_super = SuperTrack(
        name="dnase_super",
        short_label="DNAse",
        long_label="DNAse",
        visibility='pack',
    )
    for i, row in dnase.iterrows():
        dnase_track = Track(**row)
        dnase_super.add_tracks(dnase_track)
    trackdb.add_tracks(dnase_super)

In [27]:
def add_chromatin(trackdb):
    h3k27_super = SuperTrack(
        name="H3K27_super",
        short_label="H3K27ac",
        long_label="H3K27ac",
        visibility='pack',
    )
    trackdb.add_tracks(h3k27_super)

    h3k27_bed = CompositeTrack(
        name="h3k27_bed",
        short_label="H3K27ac regions",
        long_label="H3K27ac regions",
        tracktype="bigBed 9",
        dragAndDrop='subtracks',
        visibility='dense',
    )
    h3k27_super.add_tracks(h3k27_bed)

    h3k27_regions = pandas.read_excel('transfection-additional-tracks.xlsx', sheet_name='H3K27ac regions')
    for i, row in h3k27_regions.iterrows():
        h3k27_track = Track(**row, exonNumbers='off')
        h3k27_bed.add_subtrack(h3k27_track)

    h3k27_signal = CompositeTrack(
        name="h3k27_wig",
        short_label="H3K27ac signal",
        long_label="H3K27ac signal",
        tracktype="bigWig",
        dragAndDrop='subtracks',
        visibility='dense',
    )
    h3k27_super.add_tracks(h3k27_signal)

    h3k27_regions = pandas.read_excel('transfection-additional-tracks.xlsx', sheet_name='H3K27ac signal')
    for i, row in h3k27_regions.iterrows():
        h3k27_track = Track(**row, exonNumbers='off')
        h3k27_signal.add_subtrack(h3k27_track)


In [28]:
def add_wold_bed_tracks(woldsuper, bed_file_info):
    bedcomposite = CompositeTrack(
        name="woldtrans_bed",
        short_label="Transfection beds",
        long_label="Transfection track",
        tracktype="bigBed 9",
        dragAndDrop='subtracks',
        visibility='dense',
    )
    woldsuper.add_tracks(bedcomposite)

    subgroups = [
        SubGroupDefinition(
            name="cell_line",
            label="Cell Line",
            mapping=dict(ten="10T 1/2", c2="C2C12")),

        SubGroupDefinition(
            name="state",
            label="Cell State",
            mapping=dict(diff="Differentiated", 
                         mock="Mock Differentiated",
                         myoblat="Myoblat",
                         myocyte="Myocyte",
                        )),
    ]
    bedcomposite.add_subgroups(subgroups)

    bed_view = ViewTrack(
        name="bedViewTrack",
        view="Bed",
        visibility="full",
        tracktype="bigBed 9",
        short_label="Transfection",
        long_label="Transfection Beds")
    bedcomposite.add_view(bed_view)

    for filename in bed_file_info:
        rep_prefix, threshold = bed_file_info[filename]
        basename, _ = os.path.splitext(filename)
        track = Track(
            name=basename,
            short_label = rep_prefix,
            long_label = rep_prefix,
            tracktype = 'bigBed 9',
            url = os.path.join(URLBASE, GENOME, basename + '.bigBed'),
            visibility='dense',
            itemRgb='on',
            exonNumbers='off',
            #spectrum='on',
            #scoreMin=0,
            #scoreMax=maxscore,
            subgroups=dict(
                cell_line=bed_attributes[filename]['cell_line'],
                state=bed_attributes[filename]['state']))
        bed_view.add_tracks(track)
    

In [29]:
def add_wold_wiggles(woldsuper, bed_file_info, bedframes):
    make_bigwigs(bedframes)
    wig_view = ViewTrack(
        name="wigViewTrack",
        view="Wig",
        visibility="squish",
        tracktype="bigWig",
        short_label="signal",
        long_label="Transfection Signal")

    for filename in bedframes:
        basename, ext = os.path.splitext(filename)
        aveDeterm = bedframes[filename]['aveDeterm']
        min_determ = numpy.floor(aveDeterm.min())
        max_determ = numpy.ceil(aveDeterm.max())
        threshold = bed_file_info[filename][1]

        most = max(numpy.ceil(aveDeterm.quantile([.99])[.99]), threshold*2)
        
        bwname = os.path.join(URLBASE, GENOME, basename + '.bw')
        print(filename, basename, bwname)
        aggregate = AggregateTrack(
            name=basename+'_agg',
            tracktype='bigWig {} {}'.format(min_determ, max_determ),
            short_label=basename,
            long_label=basename + 'aggregate',
            visibility='full',        
            aggregate='transparentOverlay',
        )
        bw_track = Track(
            name=basename+'_signal',
            short_label = basename + '_signal',
            long_label = basename + ' signal',
            tracktype='bigWig {} {}'.format(min_determ, max_determ),
            viewLimits='{}:{}'.format(min_determ, most),
            autoScale='off',
            yLineMark = threshold,
            yLineOnOff = 'on',
            url = bwname,
            visibility='full',
            itemRgb='on',
            #color='222,235,247',
            color='147,202,225',
            exonNumbers='off',
            #spectrum='on',
            subgroups=dict(
                cell_line=bed_attributes[filename]['cell_line'],
                state=bed_attributes[filename]['state']))

        bwerr = os.path.join(URLBASE,GENOME, basename + '-error.bw')
        bwerr_track = Track(
            name=basename+'_error',
            short_label = basename + '_error',
            long_label = basename + ' error',
            tracktype='bigWig {} {}'.format(min_determ, max_determ),
            viewLimits='{}:{}'.format(min_determ, most),
            autoScale='off',
            url = bwerr,
            visibility='full',
            itemRgb='on',
            color='49,130,189',
            exonNumbers='off',
            #spectrum='on',
            subgroups=dict(
                cell_line=bed_attributes[filename]['cell_line'],
                state=bed_attributes[filename]['state']))

        aggregate.add_subtrack(bw_track)
        aggregate.add_subtrack(bwerr_track)
        #wig_view.add_tracks([aggregate])
        woldsuper.add_tracks(aggregate)

In [30]:
def build_hub():
    hub, genomes_file, genome, trackdb = default_hub(
        hub_name='transfection',
        short_label='Transfection',
        long_label='ENCODE3 Transfection Validations',
        genome='mm10',
        email='diane@caltech.edu',
    )

    add_psu_tracks(trackdb)

    woldsuper = SuperTrack(
        name="woldsuper",
        short_label="Wold-lab trans",
        long_label="Wold Lab Transfection tracks",
        visibility='dense',
    )
    trackdb.add_tracks(woldsuper)

    bedframes = generate_bed_files(annotated_transfection, BED_FILE_INFO)
    add_wold_bed_tracks(woldsuper, BED_FILE_INFO)
    add_wold_wiggles(woldsuper, BED_FILE_INFO, bedframes)
    add_dnase_track(trackdb)
    add_chromatin(trackdb)
    
    with open('/tmp/trackDb.txt', 'wt') as outstream:
        outstream.write(str(trackdb))

In [31]:
build_hub()

c2c12_myoblast_biorep1.bed c2c12_myoblast_biorep1 https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/c2c12_myoblast_biorep1.bw
c2c12_myoblast_biorep2.bed c2c12_myoblast_biorep2 https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/c2c12_myoblast_biorep2.bw
c2c12_myocyte_biorep1.bed c2c12_myocyte_biorep1 https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/c2c12_myocyte_biorep1.bw
c2c12_myocyte_biorep2.bed c2c12_myocyte_biorep2 https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/c2c12_myocyte_biorep2.bw
10T_half_biorep1.bed 10T_half_biorep1 https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/10T_half_biorep1.bw
10T_half_biorep2.bed 10T_half_biorep2 https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/10T_half_biorep2.bw
10T_half_mock_diff_biorep1.bed 10T_half_mock_diff_biorep1 https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/10T_half_mock_diff_biorep1.bw
10T_half_mock_diff_biorep2.bed 10T_half_mock_diff_biorep2 https://woldl

/usr/lib/python3/dist-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


# Are the input files close enough?

In [32]:
t1 = transfection[['#ID', 
              'C2C12_Myoblast_Rep1-ASSAY', 'C2C12_Myoblast_Rep2-ASSAY',
              'C2C12_Myocyte_Rep1-ASSAY', 'C2C12_Myocyte_Rep2-ASSAY',
              '10T1/2_Rep1-ASSAY', '10T1/2_Rep2-ASSAY',
              '10T1/2_MockDiff_Rep1-ASSAY', '10T1/2_MockDiff_Rep1-ASSAY',
             ]].copy()

In [33]:
s1 = google_sheet[['#ID', 
       'C2C12 Myoblast Bio Rep1', 'C2C12 Myoblast Bio Rep2', 
       'C2C12 Myocyte Bio Rep1', 'C2C12 Myocyte Bio Rep2',
       '10T1/2 Bio Rep1', '10T1/2 Bio Rep2',
       '10T1/2 Mock Differentiated Bio Rep1', '10T1/2 Mock Differentiated Bio Rep2'
      ]].copy()

S1 labels are nicer to read, they should be the same order, so lets repalce the column heading in t1.

In [34]:
t1.columns = s1.columns

## Are there any duplicate IDs?

In [35]:
t1[t1.duplicated(['#ID'], keep=False)]

,#ID,C2C12 Myoblast Bio Rep1,C2C12 Myoblast Bio Rep2,C2C12 Myocyte Bio Rep1,C2C12 Myocyte Bio Rep2,10T1/2 Bio Rep1,10T1/2 Bio Rep2,10T1/2 Mock Differentiated Bio Rep1,10T1/2 Mock Differentiated Bio Rep2


In [36]:
s1[s1.duplicated(['#ID'], keep=False)]

,#ID,C2C12 Myoblast Bio Rep1,C2C12 Myoblast Bio Rep2,C2C12 Myocyte Bio Rep1,C2C12 Myocyte Bio Rep2,10T1/2 Bio Rep1,10T1/2 Bio Rep2,10T1/2 Mock Differentiated Bio Rep1,10T1/2 Mock Differentiated Bio Rep2


Yes, WOLD_LR_002 shows up twice in the transfection-merged.csv

## are there vectors that are only in one file?

In [37]:
t1[t1['#ID'].isin(set(t1['#ID']).difference(s1['#ID']))]

,#ID,C2C12 Myoblast Bio Rep1,C2C12 Myoblast Bio Rep2,C2C12 Myocyte Bio Rep1,C2C12 Myocyte Bio Rep2,10T1/2 Bio Rep1,10T1/2 Bio Rep2,10T1/2 Mock Differentiated Bio Rep1,10T1/2 Mock Differentiated Bio Rep2
350,WOLD2014JAN_070,0.729393,1.900316,8.87218,8.963735,0.42702,1.0218,0.516245,0.516245


In [38]:
s1[s1['#ID'].isin(set(s1['#ID']).difference(t1['#ID']))]

,#ID,C2C12 Myoblast Bio Rep1,C2C12 Myoblast Bio Rep2,C2C12 Myocyte Bio Rep1,C2C12 Myocyte Bio Rep2,10T1/2 Bio Rep1,10T1/2 Bio Rep2,10T1/2 Mock Differentiated Bio Rep1,10T1/2 Mock Differentiated Bio Rep2
314,WOLD_LR_104,1.759908,NaN,2.844676,NaN,2.414138,NaN,1.630483,NaN


In [39]:
t1['source'] = 'transfection'

In [40]:
s1['source'] = 'google'

In [41]:
pandas.concat([t1, s1]).sort_values(by=['#ID', 'source'])

,#ID,C2C12 Myoblast Bio Rep1,C2C12 Myoblast Bio Rep2,C2C12 Myocyte Bio Rep1,C2C12 Myocyte Bio Rep2,10T1/2 Bio Rep1,10T1/2 Bio Rep2,10T1/2 Mock Differentiated Bio Rep1,10T1/2 Mock Differentiated Bio Rep2,source
0,BJW1000A,2.497751,NaN,16.201976,NaN,1.755624,NaN,1.258984,NaN,google
0,BJW1000A,0.902165,NaN,5.293485,NaN,0.624286,NaN,0.827991,0.827991,transfection
1,BJW1000B,1.653045,2.108818,10.711779,11.324074,2.082971,2.688467,8.622666,1.275765,google
1,BJW1000B,1.653045,2.108818,10.711779,11.324074,2.082971,2.688467,2.727020,2.727020,transfection
2,BJW1001,0.325529,0.211114,6.927548,14.994512,0.265127,0.397979,0.595979,0.666264,google
2,BJW1001,0.325529,0.819005,6.927548,4.898984,0.265127,0.748982,0.595979,0.595979,transfection
3,BJW1002,0.795625,0.946431,1.131369,1.075926,0.603520,1.031936,1.264036,1.449387,google
3,BJW1002,0.795625,0.946431,1.131369,1.075926,0.603520,1.031936,1.264036,1.264036,transfection
4,BJW1003,2.738967,NaN,16.857300,NaN,2.942621,NaN,2.595539,NaN,google
4,BJW1003,0.989290,NaN,5.507591,NaN,1.046373,NaN,1.706998,1.706998,transfection


In [42]:
pandas.concat([t1, s1]).sort_values(by=['#ID', 'source']).to_csv('transfection-original+merged.csv', index=False)

# Are the bigwigs correct?

In [43]:
PANDASODF = os.path.expanduser('~diane/src/pandasodf')
if PANDASODF not in sys.path:
    sys.path.append(PANDASODF)
import pandasodf

In [44]:
ods = pandasodf.ODFReader('transfection-merged.ods')
transfection_annotated = ods.parse('transfection-merged')

In [45]:
def build_location_scores(sheet, column_prefix):
    locations = {}
    reps = ['-Tech_Rep_1', '-Tech_Rep_2', '-Tech_Rep_3', '-Tech_Rep_4']
    elements = sheet[sheet['Type'].isin(('TEST ELEMENT', 'CONTROL ELEMENT'))]
    for i, row in elements.iterrows():
        name = row['#ID']
        chromosome = row['CHR(mm10)']
        if chromosome is None or chromosome.strip() == '':
            print('skipping {} no coordinates'.format(name))
            continue
        start = int(row['Start(mm10)'])
        stop = int(row['Stop(mm10)'])        
        sheet_value = row[[column_prefix+r for r in reps]].mean()
        locations.setdefault((chromosome, start, stop), []).append((name, sheet_value))
    return locations

In [46]:
def close_location(scores, bw_values):
    for name, score in scores:
        for region in bw_values:
            if numpy.allclose(score, region[2]):
                return name

In [47]:
def validate_bigwig(sheet, bigwig_url, column, verbose=0):
    stream = pyBigWig.open(bigwig_url, 'r')
    matches = 0
    mismatches = 0
    na = 0
    locations = build_location_scores(sheet, column)
    for region in locations:
        bw_values = stream.intervals(*region)
        if bw_values is None:
            na += 1
            if verbose > 1:
                print(region, locations[region])
        else:
            closest_name = close_location(locations[region], bw_values)
            if closest_name is None:
                mismatches += 1
                if verbose > 0:
                    print(region, locations[region], bw_values, 'mismatch')
                    #for bw in bw_values:
                        # get the other location names/scores by chr, start, stop
                    #    print('  ', locations[(region[0], bw[0], bw[1])])
            else:
                matches += 1
    print('{} records matched'.format(matches))
    print('{} records failed'.format(mismatches))
    print('{} records are N/A'.format(na))
    stream.close()

In [48]:
for filename, column in [
    ('10T_half_biorep1.bw', '10T1/2_Rep1'),
    ('10T_half_biorep2.bw', '10T1/2_Rep2'),
    ('10T_half_mock_diff_biorep1.bw', '10T1/2_MockDiff_Rep1'),
    ('10T_half_mock_diff_biorep2.bw', '10T1/2_MockDiff_Rep2'),
    ('c2c12_myoblast_biorep1.bw', 'C2C12_Myoblast_Rep1'),
    ('c2c12_myoblast_biorep2.bw', 'C2C12_Myoblast_Rep2'),
    ('c2c12_myocyte_biorep1.bw', 'C2C12_Myocyte_Rep1'),
    ('c2c12_myocyte_biorep2.bw', 'C2C12_Myocyte_Rep2'),

    ]:
    url = "https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/{}".format(filename)
    print('-------- {} -------------'.format(filename))
    validate_bigwig(transfection_annotated, url, column, 1)


-------- 10T_half_biorep1.bw -------------
('chr8', 123882909, 123883847) [('UP_13', 1.0502046385000001)] ((123882187, 123882932, 0.47779232263565063),) mismatch
('chr6', 108668244, 108668804) [('WOLD_LR_027', 0.9242822716666667)] ((108668244, 108668981, 1.5868430137634277),) mismatch
('chr7', 46313389, 46314120) [('WOLD_LR_045', 0.5824882876666667)] ((46313331, 46314018, 1.5047619342803955),) mismatch
297 records matched
3 records failed
1 records are N/A
-------- 10T_half_biorep2.bw -------------
('chr8', 123882187, 123882932) [('WOLD_LR_008', nan)] ((123882909, 123883847, 1.6241974830627441),) mismatch
('chr7', 46313389, 46314120) [('WOLD_LR_045', nan)] ((46313331, 46314018, 1.4308438301086426),) mismatch
257 records matched
2 records failed
42 records are N/A
-------- 10T_half_mock_diff_biorep1.bw -------------
('chr8', 123882909, 123883847) [('UP_13', 1.6520342614999999)] ((123882187, 123882932, 1.1569054126739502),) mismatch
('chr6', 108668244, 108668804) [('WOLD_LR_027', 1.88074

## Does the raw values match the expected mean and standard deviation

In [49]:
transfection.columns

Index(['#ID', 'C2C12_Myoblast_Rep1-STDEV', 'C2C12_Myoblast_Rep1-ASSAY',
       'C2C12_Myoblast_Rep1-Tech_Rep_1', 'C2C12_Myoblast_Rep1-Tech_Rep_2',
       'C2C12_Myoblast_Rep1-Tech_Rep_3', 'C2C12_Myoblast_Rep1-Tech_Rep_4',
       '10T1/2_Rep1-STDEV', '10T1/2_Rep1-ASSAY', '10T1/2_Rep1-Tech_Rep_1',
       '10T1/2_Rep1-Tech_Rep_2', '10T1/2_Rep1-Tech_Rep_3',
       '10T1/2_Rep1-Tech_Rep_4', 'C2C12_Myocyte_Rep1-STDEV',
       'C2C12_Myocyte_Rep1-ASSAY', 'C2C12_Myocyte_Rep1-Tech_Rep_1',
       'C2C12_Myocyte_Rep1-Tech_Rep_2', 'C2C12_Myocyte_Rep1-Tech_Rep_3',
       'C2C12_Myocyte_Rep1-Tech_Rep_4', '10T1/2_MockDiff_Rep1-STDEV',
       '10T1/2_MockDiff_Rep1-ASSAY', '10T1/2_MockDiff_Rep1-Tech_Rep_1',
       '10T1/2_MockDiff_Rep1-Tech_Rep_2', '10T1/2_MockDiff_Rep1-Tech_Rep_3',
       '10T1/2_MockDiff_Rep1-Tech_Rep_4', 'C2C12_Myoblast_Rep2-STDEV',
       'C2C12_Myoblast_Rep2-ASSAY', 'C2C12_Myoblast_Rep2-Tech_Rep_1',
       'C2C12_Myoblast_Rep2-Tech_Rep_2', 'C2C12_Myoblast_Rep2-Tech_Rep_3',
       

In [50]:
experiment_names = [
    'C2C12_Myoblast_Rep1', '10T1/2_Rep1', 'C2C12_Myocyte_Rep1', '10T1/2_MockDiff_Rep1',
    'C2C12_Myoblast_Rep2', '10T1/2_Rep2', 'C2C12_Myocyte_Rep2', '10T1/2_MockDiff_Rep2',
]
reps = ['-Tech_Rep_1', '-Tech_Rep_2', '-Tech_Rep_3', '-Tech_Rep_4']
rep_names = [x+y for x,y in itertools.product(experiment_names, reps)]
averages = [x+y for x,y in itertools.product(experiment_names, ['-ASSAY'])]
stdev = [x+y for x,y in itertools.product(experiment_names, ['-STDEV'])]

all_names = set(rep_names + averages + stdev)

In [51]:
set(transfection.columns).difference(all_names)

{'#ID'}

In [52]:
for experiment in experiment_names:
    replicates = transfection[[experiment+r for r in reps]]
    assay = numpy.isclose(transfection[experiment + '-ASSAY'], replicates.mean(axis=1), equal_nan=True)
    std = numpy.isclose(transfection[experiment + '-STDEV'], replicates.std(axis=1), equal_nan=True)
    print(experiment, assay.sum(), std.sum())

C2C12_Myoblast_Rep1 368 0
10T1/2_Rep1 357 1
C2C12_Myocyte_Rep1 368 0
10T1/2_MockDiff_Rep1 276 0
C2C12_Myoblast_Rep2 368 63
10T1/2_Rep2 368 63
C2C12_Myocyte_Rep2 367 62
10T1/2_MockDiff_Rep2 310 62


In [53]:
e = '10T1/2_MockDiff_Rep1'
pandas.DataFrame([
    transfection[e + '-ASSAY'], transfection[[e+r for r in reps]].mean(axis=1, skipna=True),
    transfection[e + '-STDEV'], transfection[[e+r for r in reps]].std(axis=1, skipna=True),
    numpy.isclose(transfection[e + '-ASSAY'], 
                  transfection[[e+r for r in reps]].mean(axis=1, skipna=True),
                  equal_nan=True)
]).T

,10T1/2_MockDiff_Rep1-ASSAY,Unnamed 0,10T1/2_MockDiff_Rep1-STDEV,Unnamed 1,Unnamed 2
0,0.827991,0.827991,0.226486,0.187528,1.0
1,2.727020,2.727020,0.523331,1.427134,1.0
2,0.595979,0.595979,0.142620,0.084999,1.0
3,1.264036,1.264036,0.132166,0.167062,1.0
4,1.706998,1.706998,0.168714,0.287995,1.0
5,0.798558,0.798558,0.145443,0.116144,1.0
6,0.661988,0.661988,0.214784,0.142184,1.0
7,1.606222,1.606222,0.062020,0.099618,1.0
8,1.258937,1.258937,0.321686,0.404983,1.0
9,1.488998,1.488998,0.055034,0.081946,1.0


In [54]:
numpy.mean([1.768737,2.113490])

1.9411135000000002

In [55]:
transfection['#ID'].iloc[19]

'DW_1'

In [56]:
transfection[['#ID'] + [e+r for r in reps]]

,#ID,10T1/2_MockDiff_Rep1-Tech_Rep_1,10T1/2_MockDiff_Rep1-Tech_Rep_2,10T1/2_MockDiff_Rep1-Tech_Rep_3,10T1/2_MockDiff_Rep1-Tech_Rep_4
0,BJW1000A,1.043464,0.738835,0.701673,NaN
1,BJW1000B,1.477367,4.735351,2.684810,2.010553
2,BJW1001,0.652504,0.684370,0.532625,0.514416
3,BJW1002,1.500759,1.128983,1.166920,1.259484
4,BJW1003,1.776432,1.390633,1.953928,NaN
5,BJW1004,0.836115,0.626707,0.849772,0.881639
6,BJW1005,0.666161,0.769347,0.752656,0.459788
7,BJW1006,1.558422,1.631259,1.732929,1.502276
8,BJW1007,1.726557,1.021949,1.028306,NaN
9,BJW1008,1.529590,1.543247,1.367223,1.515933


In [53]:
numpy.asarray([1,2,numpy.nan]).mean(axis=1, skipna=True)

TypeError: _mean() got an unexpected keyword argument 'skipna'

In [ ]:
replicates.std?

In [ ]:
stream  = pyBigWig.open("https://woldlab.caltech.edu/~diane/encode3-transfection/mm10/{}".format('c2c12_myocyte_biorep2.bw'))

In [ ]:
stream.intervals('chr1', 134244200, 134245800)

In [ ]:
pandas.DataFrame.quantile?

In [ ]:
pandas.Series([1,2,3]).quantile?